In [4]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
import pandas as pd

In [5]:
df = pd.read_excel("../data/processed/exploratory_analysis.xlsx")

In [6]:
feature_cols = [
    'acceptance_rate',
    'undergrad_gpa',
    'gre_quantitative_reasoning',
    'gre_verbal_reasoning',
    'analytical_writing',
    'gre_total',
    'gpa_percentile',
    'gre_avg',
    'gpa_x_acceptancerate',
    'program',          
    'degree_type',     
    'institution'        
]


In [7]:
X = df[feature_cols]
y = df['decision_grouped']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

smote = SMOTE(random_state=42)
X_train_sm, y_train_sm = smote.fit_resample(X_train, y_train)

xgb_clf = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='mlogloss')
xgb_clf.fit(X_train_sm, y_train_sm)

y_pred = xgb_clf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [21:32:44] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Accuracy: 0.5284472049689442
Classification Report:
               precision    recall  f1-score   support

           0       0.59      0.59      0.59      7440
           1       0.52      0.55      0.54      7095
           2       0.46      0.42      0.44      5590

    accuracy                           0.53     20125
   macro avg       0.52      0.52      0.52     20125
weighted avg       0.53      0.53      0.53     20125

Confusion Matrix:
 [[4375 1718 1347]
 [1723 3913 1459]
 [1370 1873 2347]]
